In [ ]:
import pandas as pd
import numpy as np

def EHF(data, tmax_col, tmin_col, date_col, p_col, n_days=3, period_days=30):
    # Verificar columnas
    required_cols = [tmax_col, tmin_col, date_col, p_col]
    if not all(col in data.columns for col in required_cols):
        raise ValueError("Error: Una o más columnas especificadas no existen en los datos.")

    # Ordenar los datos por fecha para asegurar la secuencia
    data = data.sort_values(by=date_col)

    # Calcular temperatura media ajustada (tad)
    data['TAD'] = (data[tmax_col] + data[tmin_col].shift(-1)) / 2

    # Calcular TAD_p95 por grupo y agregarlo al marco de datos original
    data['TAD_p95'] = data.groupby('com')['TAD'].transform(lambda x: x.quantile(0.95))

    # Calcular EHIsigi, EHIaccli y EHF para `TAD` y `tmax`
    data['EHIsigi_tad'] = data['TAD'].rolling(window=n_days).mean() - data['TAD_p95']
    data['EHIaccli_tad'] = data['TAD'].rolling(window=n_days).mean() - data['TAD'].rolling(window=period_days).mean()
    data['EHF_tad'] = data['EHIsigi_tad'] * np.maximum(1, data['EHIaccli_tad'])

    data['EHIsigi_tmax'] = data[tmax_col].rolling(window=n_days).mean() - data[p_col]
    data['EHIaccli_tmax'] = data[tmax_col].rolling(window=n_days).mean() - data[tmax_col].rolling(window=period_days).mean()
    data['EHF_tmax'] = data['EHIsigi_tmax'] * np.maximum(1, data['EHIaccli_tmax'])

    return data